# Generating Coordinates Script


In [ ]:
import pandas as pd
import requests
import subprocess
import json
import csv

bearer_token = "YOUR_BEARER_TOKEN"
headers = {
    "Authorization": f"Bearer {'XXXXXXX'}",
    "Content-Type": "application/json"
}

class Address:
    def __init__(self, block, street):
        self.block = block
        self.street = street
        self.coordinates = None
        self.nearest_mrt = None
        self.distance_to_mrt = None

    def __str__(self):
        return f"{self.block} {self.street}"

    def __eq__(self, other):
        if isinstance(other, Address):
            return self.block == other.block and self.street == other.street
        return False

    def __hash__(self):
        return hash((self.block, self.street))

def getCoordinates(address):
    print('Getting coordinates for : ' + address)
    url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=" + address + "&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.request("GET", url, headers=headers)
    return response.json()['results'][0]['LONGITUDE'], response.json()['results'][0]['LATITUDE']

def findNearestMRT(coordinates):
    try:
        coordinates_str = f"{coordinates[0]},{coordinates[1]}"
        result = subprocess.run(['node', 'findNearestMRT.js', coordinates_str], capture_output=True, text=True, check=True)
        nearestMRT = json.loads(result.stdout)
        mrt_name, mrt_distance = nearestMRT['result'][0]['station']['name'], nearestMRT['result'][0]['distance']
        print(f'Nearest MRT: {mrt_name}, Distance: {mrt_distance}')
        
        return mrt_name, mrt_distance
    
    except IndexError:
        print("Error finding nearest MRT")
        return None, None

def write_to_csv(address):
    with open('temp_output.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["block", "street", "longitude", "latitude"])
        # writer = csv.DictWriter(file, fieldnames=["block", "street", "longitude", "latitude", "nearest_mrt", "distance_to_mrt"])
        
        # Check if file is empty to write the header
        if file.tell() == 0:
            writer.writeheader()
        
        writer.writerow({
            "block": address.block,
            "street": address.street,
            "longitude": address.coordinates[0],
            "latitude": address.coordinates[1]
        })

def main():
    # file original.csv is obtained directly from HDB resale prices in data.gov
    file_path = 'original.csv'

    df = pd.read_csv(file_path)

    addresses = []
    
    for block, street in df[['block', 'street_name']].values:
        address = Address(block, street)
        if address not in addresses:
            addresses.append(address)
        else:
            print(f'Address {address} not appended')
    
    for address in addresses:
        address.coordinates = getCoordinates(address.block + ' ' + address.street)
        # address.nearest_mrt, address.distance_to_mrt = findNearestMRT(address.coordinates)
        print(f'Progress: {addresses.index(address) + 1}/{len(addresses)}')

        write_to_csv(address)

    print("Addresses:")
    print(addresses)
    

if __name__ == "__main__":
    main()